
<h1 align="center">SysExplorer example</h1> 

In [1]:
from cosapp.ports import Port

class GeometryPort(Port):
    def setup(self):
        self.add_variable("visible", True, desc="Should this geometry be shown?")
        self.add_variable("shape", None, desc="Geometrical object")

In [2]:
from cosapp.systems import System
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
import numpy as np
class BeamGeo(System):
    
    def setup(self):
        self.add_inward("file","path")
        self.add_inward("mesh_size",100)
        self.add_inward('width', 0.5, unit = 'm')
        self.add_inward('height',0.5, unit = 'm')
        self.add_inward('length', 5., unit = 'm')
        self.add_outward('I', desc='Second area moment')
        self.add_outward('grid', np.zeros(1))
        self.add_output(GeometryPort, 'geom')
    def compute(self):
        self.I = self.width*self.height**3/12.
        self.grid = np.array([i*self.length/self.mesh_size for i in range(0,int(self.mesh_size)+1)])
        self.geom.shape ={
            "shape": BRepPrimAPI_MakeBox(self.length, self.width, self.height).Solid(),
            "edge": True,
            "transparent": True,
            "color": "blue"
        } 

In [3]:
class BeamMeca(System):
    
    def setup(self):
        self.add_inward('E', 270, desc = "Young modulus",  valid_range = [100,150],limits = [50,200])
        self.add_inward("grid", np.zeros(101), valid_range = [2,4],limits = [-np.inf,10] )
        self.add_inward("I", 1., desc='Second area moment' )
        self.add_inward("force", -1., desc='Force value', unit = 'N' )
        self.add_inward("position", 0.5, desc="force relative position",limits= [0.1,0.9], valid_range = [0.01,0.99] )
        self.add_outward("M", np.zeros(1), desc = "Bending moments" )
        self.add_outward("Q", np.zeros(1), desc = "Shear forces" )
        self.add_outward("W", np.zeros(1), desc = "Deflections" )
        self.add_outward("maxW", 0, desc = "Max deflections" )
        self.add_outward("maxW_loc", 0, desc = "Max deflections location" )
    
    def compute(self):

        if self.position > 1.:
            position = 0.99
        elif self.position < 0:
            position = 0.
        else:
            position = self.position
        mesh_size = len(self.grid)
        L = self.grid[-1]
        b = (1.- position)*L
        a = position*L
        M =[]
        Q = []
        W = []
        for i in range(0, int(mesh_size*position)):
            x = self.grid[i]
            M.append(self.force*b*x/L)
            Q.append(self.force*b/L)
            W.append(self.force*b*x*(L**2 - b**2 - x**2)/(6*L*self.E*self.I))  
        for j in range(int(mesh_size*position), mesh_size):
            x = self.grid[j]
            M.append(self.force*a*(L - x)/L)
            Q.append(self.force*(b/L-1))
            W.append(self.force*b*x*(L**2 - b**2 - x**2)/(6*L*self.E*self.I)+ self.force*(x-a)**3/(6*self.E*self.I))
        self.M = np.array(M)
        self.Q = np.array(Q) 
        self.W = np.array(W)
        self.maxW_loc = int(np.argmax(np.absolute(self.W)))-1
        self.maxW = self.W[self.maxW_loc]


In [4]:
class Main(System):
    
    def setup(self):
        self.add_child(BeamGeo("geo"))
        self.add_child(BeamMeca("meca"))
        self.connect(self.geo.outwards, self.meca.inwards, ["I", "grid"])

In [5]:
from cosapp.drivers import NonLinearSolver, RunSingleCase
from cosapp.recorders import DataFrameRecorder

main = Main("main")
solver = main.add_driver(NonLinearSolver('solver', history = True))
solver.add_recorder(DataFrameRecorder(includes=['*']))
height_design = solver.add_child(RunSingleCase('height_design'))
height_design.design.add_unknown('geo.height').add_equation('meca.maxW == -0.02*geo.length')  
main.run_drivers()

## Start an empty dashboarh

In [6]:
from cosapp_lab.widgets import SysExplorer
a = SysExplorer( main, anchor = "tab-after")

## Start a dashboarh from template

In [7]:
a = SysExplorer( main,  template = "simple_beam.json", anchor = "tab-after")

In [8]:
main

main - Main

In [9]:
a

SysExplorer(anchor='tab-after', chart_template={'chart_template': {'modelJson': {'global': {'tabSetTabLocation…